# Cricket Analysis - Sam MacIntyre

A small toy project to perform some data exploration and prediction on cricket data (while improving my python skills)

## Data Import





In [124]:
# Library setups
import pandas as pd
import re


In [243]:
# Import match and player data
dtBatsman = pd.read_csv("Batsman_Data.csv")
dtBowlers = pd.read_csv("Bowler_data.csv")
dtGroundAverages = pd.read_csv("Ground_Averages.csv")
dtODIresults = pd.read_csv("ODI_Match_Results.csv")
dtODImatchTotals = pd.read_csv("ODI_Match_Totals.csv")
dtWCplayers = pd.read_csv("WC_players.csv")

Let's have a look at the data structure and column types:

In [156]:
dtBatsman.head(10)

,Unnamed: 0,Bat1,Runs,BF,SR,4s,6s,Opposition,Ground,Start Date,Match_ID,Batsman,Player_ID
0,1,DNB,-,-,-,-,-,v India,Nagpur,18 Dec 2009,ODI # 2933,Oshane Thomas,49619
1,2,DNB,-,-,-,-,-,v India,Kolkata,24 Dec 2009,ODI # 2935,Oshane Thomas,49619
2,3,DNB,-,-,-,-,-,v India,Delhi,27 Dec 2009,ODI # 2936,Oshane Thomas,49619
3,4,DNB,-,-,-,-,-,v Bangladesh,Dhaka,4 Jan 2010,ODI # 2937,Oshane Thomas,49619
4,5,DNB,-,-,-,-,-,v India,Dhaka,5 Jan 2010,ODI # 2938,Oshane Thomas,49619
5,6,0*,0,8,0.00,0,0,v India,Dhaka,10 Jan 2010,ODI # 2941,Oshane Thomas,49619
6,7,0*,0,0,-,0,0,v England,The Oval,28 Jun 2011,ODI # 3165,Oshane Thomas,49619
7,8,DNB,-,-,-,-,-,v England,Leeds,1 Jul 2011,ODI # 3167,Oshane Thomas,49619
8,9,DNB,-,-,-,-,-,v England,Lord's,3 Jul 2011,ODI # 3168,Oshane Thomas,49619
9,10,1*,1,3,33.33,0,0,v England,Nottingham,6 Jul 2011,ODI # 3169,Oshane Thomas,49619



A few notable things immediately:
   - **Start** **Date** is not in date-time format
   -  **Match_ID** would be more effective if converted to just a numeric
   -  **Opposition** "v" is superfluous
   -  **Runs**, **BF**, **SR**, **4s**, *and*  **6s** are not always present as a player will not always bat in an ODI
   -  Each row represents 1 match
   - There is an unnecessary index column
 

In [157]:
dtBowlers.head(10)

,Unnamed: 0,Overs,Mdns,Runs,Wkts,Econ,Ave,SR,Opposition,Ground,Start Date,Match_ID,Bowler,Player_ID
0,1,8.0,0,57,0,7.12,-,-,v India,Nagpur,18 Dec 2009,ODI # 2933,Suranga Lakmal,49619
1,2,10.0,0,55,2,5.50,27.50,30.0,v India,Kolkata,24 Dec 2009,ODI # 2935,Suranga Lakmal,49619
2,3,-,-,-,-,-,-,-,v India,Delhi,27 Dec 2009,ODI # 2936,Suranga Lakmal,49619
3,4,9.0,1,63,2,7.00,31.50,27.0,v Bangladesh,Dhaka,4 Jan 2010,ODI # 2937,Suranga Lakmal,49619
4,5,8.0,1,48,0,6.00,-,-,v India,Dhaka,5 Jan 2010,ODI # 2938,Suranga Lakmal,49619
5,6,10.0,0,75,0,7.50,-,-,v India,Dhaka,10 Jan 2010,ODI # 2941,Suranga Lakmal,49619
6,7,7.0,0,52,2,7.42,26.00,21.0,v England,The Oval,28 Jun 2011,ODI # 3165,Suranga Lakmal,49619
7,8,7.5,0,43,3,5.48,14.33,15.6,v England,Leeds,1 Jul 2011,ODI # 3167,Suranga Lakmal,49619
8,9,10.0,0,62,2,6.20,31.00,30.0,v England,Lord's,3 Jul 2011,ODI # 3168,Suranga Lakmal,49619
9,10,2.0,0,12,0,6.00,-,-,v England,Nottingham,6 Jul 2011,ODI # 3169,Suranga Lakmal,49619


Similar comments as above apply, questions arise again about how to deal with the missing data?
Does it signify that the bowler was playing but unused? Is this useful data in itself?


In [158]:
dtGroundAverages.head()

,Ground,Span,Mat,Won,Tied,NR,Runs,Wkts,Balls,Ave,RPO
0,"Eden Gardens, Kolkata - India",2013-2017,4,4,0,0,2161,72,2297,30.01,5.64
1,"Feroz Shah Kotla, Delhi - India",2013-2019,4,4,0,0,1789,75,2331,23.85,4.60
2,Melbourne Cricket Ground - Australia,2013-2019,15,15,0,0,7656,217,8482,35.28,5.41
3,"Saurashtra Cricket Association Stadium, Rajkot...",2013-2015,2,2,0,0,1163,26,1200,44.73,5.81
4,Adelaide Oval - Australia,2013-2019,10,10,0,0,4863,157,5645,30.97,5.16


Comments:
   - **Span** variable is not in a useful format (YearStart, YearEnd could be better)
   - Need to be careful with using such sparse data over long periods of time (does an average over 4 results over 4years give a real idea of current scoring nature of the pitch?

In [159]:
dtODIresults.head()

,Unnamed: 0,Result,Margin,BR,Toss,Bat,Opposition,Ground,Start Date,Match_ID,Country,Country_ID
0,418,won,85 runs,NaN,lost,1st,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7
1,692,lost,85 runs,NaN,won,2nd,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6
2,419,lost,10 runs,NaN,lost,2nd,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7
3,693,won,10 runs,NaN,won,1st,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6
4,121,lost,107 runs,NaN,lost,2nd,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8


Comments:
- First column is a unique ID
- Country is the home team
- Only the 10 world cup teams are included in the dataset
- **Margin** should be converted into a numeric variable
- **Start Date** --> date-time
- **Result**/**Toss** should be converted to numeric/boolean
- **Bat 1st** can be a dummy variable

In [160]:
dtODImatchTotals.head()

,Unnamed: 0,Score,Overs,RPO,Target,Inns,Result,Opposition,Ground,Start Date,Match_ID,Country,Country_ID
0,412,250,48.3,5.15,NaN,1,won,v India,Kolkata,3 Jan 2013,ODI # 3315,Pakistan,7
1,680,165,48.0,3.43,251.0,2,lost,v Pakistan,Kolkata,3 Jan 2013,ODI # 3315,India,6
2,413,157,48.5,3.21,168.0,2,lost,v India,Delhi,6 Jan 2013,ODI # 3316,Pakistan,7
3,681,167,43.4,3.82,NaN,1,won,v Pakistan,Delhi,6 Jan 2013,ODI # 3316,India,6
4,117,198,40.0,4.95,306.0,2,lost,v Australia,Melbourne,11 Jan 2013,ODI # 3317,SriLanka,8


Comments:
- Contains the scores of the team in the **Country** column
- **Target** only applicable if the team is chasing
- **Inns** = 1 will always imply NaN for **Target** as the team is batting first
- There are many opportunities to reduce the number of columns by merging these dataets

In [161]:
dtWCplayers.head()

,Player,ID,Country
0,Gulbadin Naib (c),352048,Afghanistan
1,Rashid Khan (vc),793463,Afghanistan
2,Aftab Alam,440963,Afghanistan
3,Asghar Afghan,320652,Afghanistan
4,Dawlat Zadran,516561,Afghanistan


Comments:
    List of all world cup players and their associated country

## Data Pre-Processing

In [244]:
# Convert Match_ID to string
dtBatsman['Match_ID'] = dtBatsman['Match_ID'].astype(str)

# Split ID on spaces and take only numeric part
new = dtBatsman["Match_ID"].str.split(" ", n = 2, expand = True) 

# Define new columns with match type and match ID
dtBatsman['Match_Type'] = new[0]
dtBatsman['Match_ID'] = new[2].astype(int)


In [245]:

set(dtBatsman['Match_Type'])

{'ODI'}

We can see that ODI is the only match type (which was expected but good to verify), let's remove this column for cleanliness.

In [246]:
dtBatsman = dtBatsman.drop(columns = 'Match_Type')

In [248]:
# Remove 'v ' from Opposition
dtBatsman['Opposition'] = dtBatsman['Opposition'].apply(lambda x: x.split(" ", 1)[1])

Let's remove rows where the player didn't bat

In [254]:
index = dtBatsman[(dtBatsman.Bat1 == 'DNB') | (dtBatsman.Bat1 == 'TDNB') ].index
dtBatsman.drop(index, inplace = True)

Note the presence of absent/sub, absent should be removed.

In [256]:
indexes = dtBatsman[dtBatsman.Bat1 == 'absent'].index
dtBatsman.drop(indexes, inplace = True)

In [257]:
indexes = dtBatsman[dtBatsman.Bat1 == 'sub'].index
dtBatsman.drop(indexes, inplace = True)

In [258]:
set(dtBatsman.Bat1)

{'0',
 '0*',
 '1',
 '1*',
 '10',
 '10*',
 '100',
 '100*',
 '101',
 '101*',
 '102',
 '102*',
 '103',
 '103*',
 '104',
 '104*',
 '105',
 '105*',
 '106',
 '106*',
 '107',
 '107*',
 '108',
 '108*',
 '109',
 '109*',
 '11',
 '11*',
 '110',
 '110*',
 '111',
 '111*',
 '112',
 '112*',
 '113',
 '113*',
 '114',
 '115',
 '115*',
 '116',
 '116*',
 '117',
 '117*',
 '118',
 '118*',
 '119',
 '119*',
 '12',
 '12*',
 '120',
 '121',
 '122',
 '122*',
 '123',
 '123*',
 '124',
 '124*',
 '125',
 '125*',
 '126',
 '127',
 '127*',
 '128',
 '128*',
 '129',
 '129*',
 '13',
 '13*',
 '130',
 '130*',
 '131',
 '131*',
 '132',
 '132*',
 '133',
 '133*',
 '134',
 '134*',
 '135',
 '136',
 '136*',
 '137',
 '137*',
 '138',
 '138*',
 '139',
 '139*',
 '14',
 '14*',
 '140',
 '140*',
 '141*',
 '143',
 '144',
 '145*',
 '146*',
 '147',
 '148',
 '149',
 '15',
 '15*',
 '150',
 '150*',
 '151',
 '152',
 '152*',
 '153*',
 '154',
 '154*',
 '156',
 '157*',
 '159',
 '16',
 '16*',
 '160*',
 '162',
 '163',
 '164',
 '168*',
 '17',
 '17*',


In [259]:
dtBatsman = dtBatsman.astype({'Runs':int, 'BF':int, '4s':int, '6s':int, 'Opposition':str, 'Ground':str})


In [226]:
dtBatsman.dtypes

Unnamed: 0     int64
Bat1          object
Runs           int64
BF             int64
SR            object
4s             int64
6s             int64
Opposition    object
Ground        object
Start Date    object
Match_ID       int64
Batsman       object
Player_ID      int64
dtype: object

In [260]:

# Remove all entries with 0 balls faced
dtBatsman = dtBatsman.drop(dtBatsman[dtBatsman.BF == 0].index)

In [261]:
dtBatsman.SR = dtBatsman.SR.astype(float)

In [262]:
dtBatsman.dtypes

Unnamed: 0      int64
Bat1           object
Runs            int64
BF              int64
SR            float64
4s              int64
6s              int64
Opposition     object
Ground         object
Start Date     object
Match_ID        int64
Batsman        object
Player_ID       int64
dtype: object

In [263]:
dtBatsman

,Unnamed: 0,Bat1,Runs,BF,SR,4s,6s,Opposition,Ground,Start Date,Match_ID,Batsman,Player_ID
5,6,0*,0,8,0.00,0,0,India,Dhaka,10 Jan 2010,2941,Oshane Thomas,49619
9,10,1*,1,3,33.33,0,0,England,Nottingham,6 Jul 2011,3169,Oshane Thomas,49619
10,11,0*,0,2,0.00,0,0,Australia,Pallekele,10 Aug 2011,3175,Oshane Thomas,49619
11,12,0,0,2,0.00,0,0,Pakistan,Dubai (DSC),11 Nov 2011,3212,Oshane Thomas,49619
13,14,0,0,4,0.00,0,0,Pakistan,Dhaka,15 Mar 2012,3260,Oshane Thomas,49619
14,15,0,0,1,0.00,0,0,Bangladesh,Dhaka,20 Mar 2012,3265,Oshane Thomas,49619
15,16,1,1,5,20.00,0,0,India,Port of Spain,11 Jul 2013,3388,Oshane Thomas,49619
17,18,1*,1,1,100.00,0,0,Pakistan,Sharjah,18 Dec 2013,3445,Oshane Thomas,49619
18,19,2*,2,11,18.18,0,0,Pakistan,Abu Dhabi,25 Dec 2013,3448,Oshane Thomas,49619
26,27,4*,4,5,80.00,0,0,England,The Oval,22 May 2014,3492,Oshane Thomas,49619


Still to do:
- Remove first column
- Convert date to date-time object
- Create a "not-out" dummy variable